## INSOFE Transfer Learning Assignment

### VGG16 For Comparison

#### Data Extraction

In [18]:
import os

data_path =  'data'
data_dir_list = os.listdir(data_path)

data_dir_list = [x for x in data_dir_list if x not in ['.DS_Store', '.floydexpt', '.floydignore','.floyddata']]
print(data_dir_list)

['dogs', 'cats', 'Humans', 'horses']


In [19]:
img_rows=224
img_cols=224
num_channel=3
num_epoch=2

img_data_list=[]
classes_names_list=[]

In [20]:
import cv2

for dataset in data_dir_list:
    classes_names_list.append(dataset) 
    print ('Loading images from {} folder\n'.format(dataset)) 
    img_list = os.listdir(data_path +'/'+ dataset)
    for img in img_list:
        input_img = cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img_resize = cv2.resize(input_img,(img_rows, img_cols))
        img_data_list.append(input_img_resize)

Loading images from dogs folder

Loading images from cats folder

Loading images from Humans folder

Loading images from horses folder



In [21]:
num_classes = len(classes_names_list)

#### Data preprocessing

In [22]:
import numpy as np
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255

img_data = img_data.reshape(img_data.shape[0], img_data.shape[1], img_data.shape[2], num_channel)

In [23]:
num_of_samples = img_data.shape[0]
input_shape = img_data[0].shape

In [24]:
classes = np.ones((num_of_samples,), dtype='int64')
classes[0:202]=0
classes[202:404]=1
classes[404:606]=2
classes[606:]=3

In [25]:
from keras.utils import to_categorical

# convert class labels to on-hot encoding
classes = to_categorical(classes, num_classes)

In [26]:
from sklearn.utils import shuffle

x, y = shuffle(img_data, classes, random_state=2)

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

In [28]:
from keras.layers.core import Dense
from keras.layers import Input

image_input = Input(shape=(224, 224, 3))

#### Model Building

In [5]:
from keras.applications.vgg16 import VGG16

model = VGG16(input_tensor=image_input, include_top=True, weights='imagenet')

model.summary()

553435136/553467096 [============================>.] - ETA: 0s_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (M

In [21]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

In [22]:
last_layer = model.get_layer('block5_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
out = Dense(num_classes, activation='softmax', name='output')(x)
custom_vgg_model2 = Model(image_input, out)

In [23]:
custom_vgg_model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [24]:
# freeze all the layers except the dense layers
for layer in custom_vgg_model2.layers[:-3]:
    layer.trainable = False

In [25]:
custom_vgg_model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [26]:
custom_vgg_model2.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])

In [ ]:
hist = custom_vgg_model2.fit(X_train, y_train, batch_size=32, epochs=8, verbose=1, validation_data=(X_test, y_test))

Train on 646 samples, validate on 162 samples
Epoch 1/8
640/646 [============================>.] - ETA: 5s - loss: 1.2126 - acc: 0.5016 

In [ ]:
(loss, accuracy) = custom_vgg_model2.evaluate(X_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

### VGG 19 Model with Adadelta and Categorical Crossentropy

In [ ]:
from keras.applications.vgg19 import VGG19

model = VGG19(input_tensor=image_input, include_top=True, weights='imagenet')

model.summary()

502931456/574710816 [=========================>....] - ETA: 65s

In [ ]:
last_layer = model.get_layer('block5_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
out = Dense(num_classes, activation='softmax', name='output')(x)
custom_vgg19_model = Model(image_input, out)

In [ ]:
model.summary()

In [ ]:
for layer in custom_vgg19_model.layers[:-3]:
    layer.trainable = False

In [ ]:
custom_vgg_model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])

In [ ]:
hist = custom_vgg19_model.fit(X_train, y_train, batch_size=32, epochs=8, verbose=1, validation_data=(X_test, y_test))

In [ ]:
(loss, accuracy) = custom_vgg19_model.evaluate(X_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

### ResNet Model with Adadelta and Categorical Crossentropy

In [ ]:
from keras.applications.resnet50 import ResNet50

modelRes = ResNet50(input_tensor=image_input, include_top=True, weights='imagenet')

modelRes.summary()

In [ ]:
last_layer = modelRes.get_layer('block5_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
out = Dense(num_classes, activation='softmax', name='output')(x)
custom_res_model = Model(image_input, out)

In [ ]:
modelRes.summary()

In [ ]:
for layer in custom_res_model.layers[:-3]:
    layer.trainable = False

In [ ]:
custom_res_model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [ ]:
hist = custom_res_model.fit(X_train, y_train, batch_size=32, epochs=8, verbose=1, validation_data=(X_test, y_test))

In [ ]:
(loss, accuracy) = custom_res_model.evaluate(X_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

### InceptionV3 Model with Adadelta and Categorical Crossentropy

In [29]:
from keras.applications import InceptionV3

modelV3 = InceptionV3(input_tensor=image_input, include_top=True, weights='imagenet')

modelV3.summary()

96043008/96112376 [============================>.] - ETA: 0s ETA____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 111, 111, 32)  864         input_2[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 111, 111, 32)  96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 111, 111, 32)  0           batch_normalization_1[0][0]      
___________________________

In [31]:
last_layer = modelV3.get_layer('avg_pool').output
x = Dense(128, activation='relu', name='predictions')(x)
out = Dense(num_classes, activation='softmax', name='output')(x)
custom_v3_model = Model(image_input, out)

ValueError: Layer predictions was called with an input that isn't a symbolic tensor. Received type: <class 'numpy.ndarray'>. Full input: [array([[[[0.7019608 , 0.50980395, 0.627451  ],
         [0.68235296, 0.53333336, 0.6156863 ],
         [0.6862745 , 0.5529412 , 0.61960787],
         ...,
         [0.06666667, 0.09019608, 0.09019608],
         [0.10196079, 0.1254902 , 0.1254902 ],
         [0.05882353, 0.0627451 , 0.07843138]],

        [[0.7137255 , 0.52156866, 0.6392157 ],
         [0.6901961 , 0.54509807, 0.62352943],
         [0.6862745 , 0.5529412 , 0.61960787],
         ...,
         [0.0627451 , 0.08627451, 0.08235294],
         [0.10196079, 0.12156863, 0.12156863],
         [0.05490196, 0.05882353, 0.07450981]],

        [[0.7254902 , 0.53333336, 0.6509804 ],
         [0.7019608 , 0.5529412 , 0.63529414],
         [0.6901961 , 0.5568628 , 0.62352943],
         ...,
         [0.07450981, 0.09411765, 0.09803922],
         [0.11372549, 0.12941177, 0.13333334],
         [0.06274

In [ ]:
modelV3.summary()

In [ ]:
for layer in custom_v3_model.layers[:-1]:
    layer.trainable = False

In [ ]:
custom_v3_model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [ ]:
hist = custom_v3_model.fit(X_train, y_train, batch_size=32, epochs=8, verbose=1, validation_data=(X_test, y_test))

In [ ]:
(loss, accuracy) = custom_v3_model.evaluate(X_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))